In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from gensim import corpora
from gensim.models import LdaModel
import pandas as pd

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
df = pd.read_csv('npr.csv')
documents = df['Article'].tolist()

In [3]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalnum()]
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

preprocessed_documents = [preprocess_text(doc) for doc in documents]
print(preprocessed_documents[0])

['washington', '2016', 'even', 'policy', 'bipartisan', 'politics', 'sense', 'year', 'show', 'little', 'sign', 'ending', 'president', 'obama', 'moved', 'sanction', 'russia', 'alleged', 'interference', 'election', 'concluded', 'republican', 'long', 'called', 'similar', 'severe', 'measure', 'could', 'scarcely', 'bring', 'approve', 'house', 'speaker', 'paul', 'ryan', 'called', 'obama', 'measure', 'appropriate', 'also', 'overdue', 'prime', 'example', 'administration', 'ineffective', 'foreign', 'policy', 'left', 'america', 'weaker', 'eye', 'gop', 'leader', 'sounded', 'much', 'theme', 'urging', 'president', 'obama', 'year', 'take', 'strong', 'action', 'deter', 'russia', 'worldwide', 'aggression', 'including', 'operation', 'wrote', 'devin', 'nunes', 'chairman', 'house', 'intelligence', 'committee', 'week', 'left', 'office', 'president', 'suddenly', 'decided', 'stronger', 'measure', 'indeed', 'appearing', 'cnn', 'frequent', 'obama', 'critic', 'trent', 'frank', 'called', 'much', 'tougher', 'acti

In [4]:
#create a gensim dictionary object from the preprocessed doc
dictionary = corpora.Dictionary(preprocessed_documents)

#filter out tokens that appear in less than 15 doc or more than 50% of the doc
dictionary.filter_extremes(no_below=15, no_above=0.5)

#convert each preprocessed document into a BoW representation using the dictionary
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_documents]

In [5]:
#Run LDA
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

In [6]:
#empty list to store dominiant topic labels for each doc
article_labels = []

#iterate over each preprocessed document
for i, doc in enumerate(preprocessed_documents):
    #for each doc, convert to box representation
    bow = dictionary.doc2bow(doc)
    
    #get list of topic probabilities
    topics = lda_model.get_document_topics(bow)
    
    #determine the topic with highest probability
    dominant_topic = max(topics, key=lambda x: x[1])[0]
    
    #append to the list
    article_labels.append(dominant_topic)
    
#create dataframe
df_result = pd.DataFrame({"Article":documents, "Topic":article_labels})

#print dataframe
print("Table with Articles and Topic:")
print(df_result)
print()

Table with Articles and Topic:
                                                 Article  Topic
0      In the Washington of 2016, even when the polic...      1
1        Donald Trump has used Twitter  —   his prefe...      1
2        Donald Trump is unabashedly praising Russian...      1
3      Updated at 2:50 p. m. ET, Russian President Vl...      1
4      From photography, illustration and video, to d...      0
...                                                  ...    ...
11987  The number of law enforcement officers shot an...      2
11988    Trump is busy these days with victory tours,...      1
11989  It’s always interesting for the Goats and Soda...      4
11990  The election of Donald Trump was a surprise to...      1
11991  Voters in the English city of Sunderland did s...      2

[11992 rows x 2 columns]



In [7]:
#print top terms for each topic
for topic_id in range(lda_model.num_topics):
    print(f"Top terms for Topic #{topic_id}:")
    top_terms = lda_model.show_topic(topic_id, topn=10)
    print([term[0] for term in top_terms])
    print()

Top terms for Topic #0:
['know', 'think', 'thing', 'life', 'really', 'story', 'woman', 'world', 'show', 'back']

Top terms for Topic #1:
['trump', 'clinton', 'president', 'state', 'republican', 'campaign', 'election', 'obama', 'vote', 'house']

Top terms for Topic #2:
['police', 'country', 'report', 'government', 'city', 'state', 'attack', 'told', 'officer', 'two']

Top terms for Topic #3:
['company', 'food', 'state', 'million', 'percent', 'water', '000', 'health', 'plan', 'money']

Top terms for Topic #4:
['school', 'health', 'child', 'study', 'student', 'woman', 'university', 'patient', 'drug', 'disease']



In [8]:
#print the top terms for each topic with weight
print("Top Terms for Each Topic:")
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx}:")
    terms = [term.strip() for term in topic.split("+")]
    
    for term in terms:
        weight, word = term.split("*")
        print(f"- {word.strip()} (weight: {weight.strip()})")
    print()

Top Terms for Each Topic:
Topic 0:
- "know" (weight: 0.005)
- "think" (weight: 0.005)
- "thing" (weight: 0.005)
- "life" (weight: 0.004)
- "really" (weight: 0.004)
- "story" (weight: 0.004)
- "woman" (weight: 0.003)
- "world" (weight: 0.003)
- "show" (weight: 0.003)
- "back" (weight: 0.003)

Topic 1:
- "trump" (weight: 0.027)
- "clinton" (weight: 0.011)
- "president" (weight: 0.010)
- "state" (weight: 0.009)
- "republican" (weight: 0.007)
- "campaign" (weight: 0.007)
- "election" (weight: 0.006)
- "obama" (weight: 0.005)
- "vote" (weight: 0.005)
- "house" (weight: 0.004)

Topic 2:
- "police" (weight: 0.007)
- "country" (weight: 0.006)
- "report" (weight: 0.005)
- "government" (weight: 0.004)
- "city" (weight: 0.004)
- "state" (weight: 0.004)
- "attack" (weight: 0.004)
- "told" (weight: 0.004)
- "officer" (weight: 0.004)
- "two" (weight: 0.004)

Topic 3:
- "company" (weight: 0.008)
- "food" (weight: 0.005)
- "state" (weight: 0.005)
- "million" (weight: 0.005)
- "percent" (weight: 0.005)